<a href="https://colab.research.google.com/github/Khaled-Mohammed-Abdelgaber/ml-projects-for-training/blob/main/1projectk_mean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pywaffle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.1 MB 43.2 MB/s 


In [2]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
from pywaffle import Waffle

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import statistics
from itertools import product
from scipy.cluster import hierarchy

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
df = pd.read_csv("german_credit_data - german_credit_data.csv")
df

,CustomerID,Age,Gender,Job,Duration
0,0,67,male,2,6
1,1,22,female,2,48
2,2,49,male,1,12
3,3,45,male,2,42
4,4,53,male,2,24
...,...,...,...,...,...
995,995,31,female,1,12
996,996,40,male,3,30
997,997,38,male,2,12
998,998,23,male,2,45


In [5]:
df = df.drop(['CustomerID'], axis = 1)
df['Job k$'] = 1000 * df['Job']
df.rename(columns = {'Job (k$)': 'Job', 'Duration (1-100)': 'Duration'}, inplace = True)
print('There are {} missing values in the dataframe'.format(df.isna().sum().sum()))
df.head()

There are 0 missing values in the dataframe


,Age,Gender,Job,Duration,Job k$
0,67,male,2,6,2000
1,22,female,2,48,2000
2,49,male,1,12,1000
3,45,male,2,42,2000
4,53,male,2,24,2000


In [6]:
dummies = pd.get_dummies(df.Gender)
 
# Concatenate the dummies to original dataframe
merged = pd.concat([df, dummies], axis='columns')
merged.head()

,Age,Gender,Job,Duration,Job k$,female,male
0,67,male,2,6,2000,0,1
1,22,female,2,48,2000,1,0
2,49,male,1,12,1000,0,1
3,45,male,2,42,2000,0,1
4,53,male,2,24,2000,0,1


In [7]:
df = merged.drop(['Gender','female'], axis='columns') 

In [8]:
df['Gender'] = df['male']
df.drop(['male'],axis = 'columns',inplace = True)
df.head()

,Age,Job,Duration,Job k$,Gender
0,67,2,6,2000,1
1,22,2,48,2000,0
2,49,1,12,1000,1
3,45,2,42,2000,1
4,53,2,24,2000,1


In [9]:
summary = pd.DataFrame(df.describe())

In [10]:
summary

,Age,Job,Duration,Job k$,Gender
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,20.903000,1904.000000,0.690000
std,11.375469,0.653614,12.058814,653.613962,0.462725
min,19.000000,0.000000,4.000000,0.000000,0.000000
25%,27.000000,2.000000,12.000000,2000.000000,0.000000
50%,33.000000,2.000000,18.000000,2000.000000,1.000000
75%,42.000000,2.000000,24.000000,2000.000000,1.000000
max,75.000000,3.000000,72.000000,3000.000000,1.000000


In [11]:
summary = pd.DataFrame(df.describe())
summary = summary.style.background_gradient(cmap = 'Blues') \
          .set_table_attributes("style = 'display: inline'") \
          .set_caption('Statistics of the Dataset') \
          .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
          }])
summary

,Age,Job,Duration,Job k$,Gender
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,20.903000,1904.000000,0.690000
std,11.375469,0.653614,12.058814,653.613962,0.462725
min,19.000000,0.000000,4.000000,0.000000,0.000000
25%,27.000000,2.000000,12.000000,2000.000000,0.000000
50%,33.000000,2.000000,18.000000,2000.000000,1.000000
75%,42.000000,2.000000,24.000000,2000.000000,1.000000
max,75.000000,3.000000,72.000000,3000.000000,1.000000


In [35]:
Waffle

pywaffle.waffle.Waffle

In [36]:
gender = df['Gender'].value_counts()

fig = plt.figure(FigureClass = Waffle)
#, 
"""rows = 4,
    columns = 8,
    values = gender,
    colors = ('#B13167', '#3274A1'),
    labels = ['{} - {}'.format(a, b) for a, b in zip(gender.index, gender)],
    legend = {
        'loc': 'upper left', 
        'bbox_to_anchor': (1, 1), 
        'fontsize': 15, 
        'labelcolor': 'linecolor',
        'title': 'Gender',
        'title_fontsize': 15,
        'facecolor': 'white'
        },
    font_size = 75, 
    icons = ['venus', 'mars'],
    icon_legend = True,
    figsize = (15, 8),
    facecolor = 'black'
)

plt.title('Gender Distribution', fontsize = 25, color = 'white')
plt.show()"""

ValueError: ignored

In [18]:
def generate_plotly_distributions_by_gender(feature_name):
    
    fig = px.violin(
                    df, 
                    x = 'Gender', 
                    y = feature_name, 
                    color = 'Gender', 
                    box = True, 
                    points = 'all',
                    template = 'plotly_dark'
    )

    fig.update_layout(
         autosize = False,
         width = 800,
         height = 800,
         title = dict(
             text = '<b> Distribution of ' + feature_name + ' by Gender</b>',
             x = 0.5,
             y = 0.95,
             font = dict(
                 size = 20
             )
         ),
         legend_title = '<b>Gender</b>',
         xaxis_title = '<b>Gender</b>',
         yaxis_title = '<b> ' + feature_name + ' </b>'
    )

    fig.show()
    
generate_plotly_distributions_by_gender(df.columns[1])
generate_plotly_distributions_by_gender(df.columns[2])


In [19]:
def generate_plotly_distributions_by_gender(feature_name):
   
    fig = px.violin(
                    df.any(),
                    x = 'Gender',
                    y = feature_name,
                    color = 'Gender',
                    box = True,
                    points = 'all',
                    template = 'plotly_dark'
    )

    fig.update_layout(
         autosize = False,
         width = 800,
         height = 800,
         title = dict(
             text = '<b> Distribution of ' + feature_name + ' by Gender</b>',
             x = 0.5,
             y = 0.95,
             font = dict(
                 size = 20
             )
         ),
         legend_title = '<b>Gender</b>',
         xaxis_title = '<b>Gender</b>',
         yaxis_title = '<b> ' + feature_name + ' </b>'
    )

    fig.show()
   
generate_plotly_distributions_by_gender(df.columns[1])
generate_plotly_distributions_by_gender(df.columns[2])
generate_plotly_distributions_by_gender(df.columns[3])



ValueError: ignored

In [20]:
df['Job'] =  df['Job'] /1000

df['Duration'] =  df['Duration'] /10
fig = px.histogram(df, x = 'Age', y = 'Duration',
                   color = 'Gender', color_discrete_sequence = ['#2D46B9','#F037A5'],
                   marginal = 'box', hover_data = df.columns,
                   template = 'plotly_dark')

fig = px.histogram(df, x = 'Job', y = 'Duration', 
                   color = 'Gender', color_discrete_sequence = ['#87BE7A', '#0D782A'], 
                   marginal = 'box', hover_data = df.columns,
                   template = 'plotly_dark')


fig.update_layout(
         autosize = False,
         width = 800,
         height = 800,
         title = dict(
             text = '<b>Sum of Duration vs Age for Men and Women</b>',
             x = 0.5,
             y = 0.95,
             font = dict(
                 size = 20
             )
         ),
         legend_title = '<b>Gender</b>',
         xaxis_title = '<b>Age</b>',
         yaxis_title = '<b>Sum of Duration</b>'
)

fig.show()

In [21]:
fig = ff.create_scatterplotmatrix(df, diag = 'box', index = 'Gender', colormap = ['#FFFF01', '#8D00CB'])

fig.update_layout(
    template = 'plotly_dark',
    yaxis_range = [1, 120],
    title = dict(
         text = '<b>Scatter Plot for the Dataset</b>',
         x = 0.5,
         y = 0.95,
         font = dict(
             size = 20
         )
    ),
    width = 800,
    height = 800)

fig.update_xaxes(showgrid = False, gridcolor = 'black')
fig.update_yaxes(showgrid = False, gridcolor = 'black')
fig.show()


In [22]:
correlations = df.corr()
x = correlations.columns.tolist()
y = correlations.index.tolist()
z = correlations.values

fig = ff.create_annotated_heatmap(x = x, y = y, z = z, 
                                  annotation_text = z.round(2), 
                                  colorscale = 'teal',
                                  hovertemplate = "Correlation of %{x} and %{y}= %{z:.2f}")

fig.update_layout(
    title = dict(
         text = '<b>Feature Correlation</b>',
         x = 0.5,
         y = 0.95,
         font = dict(
             size = 20
         )
    ),
    template = 'plotly_dark',
    width = 800,
    height = 800
)

fig.show()

In [23]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
X = df
print(X)

     Age    Job  Duration  Job k$  Gender
0     67  0.002       0.6    2000       1
1     22  0.002       4.8    2000       0
2     49  0.001       1.2    1000       1
3     45  0.002       4.2    2000       1
4     53  0.002       2.4    2000       1
..   ...    ...       ...     ...     ...
995   31  0.001       1.2    1000       0
996   40  0.003       3.0    3000       1
997   38  0.002       1.2    2000       1
998   23  0.002       4.5    2000       1
999   27  0.002       4.5    2000       1

[1000 rows x 5 columns]


In [24]:
df['Job'] = df['Job'] / 1000 # reduce Y-axis from B to K


In [25]:
distortion = {}

for i in range(1, 16):
    km = KMeans(n_clusters = i, init = 'k-means++', random_state = 0).fit(X.values.reshape(-1,1))
    distortion.update({i: km.inertia_})

number_of_clusters = list(distortion.keys())
inertia = list(distortion.values())
    
fig = px.line(x = number_of_clusters, y = inertia, markers = True)

fig.update_layout(template = 'plotly_dark',
                  width = 800, 
                  height = 800,
                  title = dict(
                  text = '<b>Elbow Curve</b>',
                     x = 0.5,
                     y = 0.95,
                     font = dict(
                         size = 20
                         )
                  ),
                  xaxis_title = '<b>Number of Clusters</b>',
                  yaxis_title = '<b>Inertia</b>'
)

fig.update_traces(marker = dict(
    size = 7, 
    color = '#B7E0F7')
)

fig.add_annotation(x = 5, y = 75000, 
                   showarrow = True, 
                   text = 'Optimal Clusters', 
                   arrowhead = 1, 
                   arrowsize = 2,
                   ax = 7,
                   ay = -100,
                   bordercolor = '#FFFF01',
                   borderpad = 3
                  )

fig.show()

In [26]:
km = KMeans(n_clusters = 5, random_state = 0)
cluster_predictions = km.fit_predict(X)

df['Cluster Segregation'] = cluster_predictions
df = df.sort_values(by = 'Cluster Segregation')
df['Cluster Segregation'] = df['Cluster Segregation'].astype(str)

fig = px.scatter(df, x = 'Job', 
                 y = 'Duration', 
                 color = 'Cluster Segregation', 
                 color_discrete_sequence = px.colors.qualitative.Set2)

fig.update_traces(marker = dict(size = 10, opacity = 0.80))

fig.update_layout(
    template = 'plotly_dark',
    width = 800,
    legend_title = 'Clusters',
    title = dict(
             text = '<b>K-Means Clustering</b>',
             x = 0.5,
             y = 0.95,
             font = dict(
                 size = 20
             )
         ),
         xaxis_title = '<b>Annual Income</b>',
         yaxis_title = '<b>Sum of Spending Scores</b>'
)

fig.show()

In [27]:
km = KMeans(n_clusters = 5, random_state = 0)
cluster_predictions = km.fit_predict(X)

df['Cluster Segregation'] = cluster_predictions
df = df.sort_values(by = 'Cluster Segregation')
df['Cluster Segregation'] = df['Cluster Segregation'].astype(str)

fig = px.scatter(df, x = 'Job', 
                 y = 'Duration', 
                 color = 'Cluster Segregation', 
                 color_discrete_sequence = px.colors.qualitative.Set2)

fig.update_traces(marker = dict(size = 10, opacity = 0.80))

fig.update_layout(
    template = 'plotly_dark',
    width = 800,
    legend_title = 'Clusters',
    title = dict(
             text = '<b>K-Means Clustering</b>',
             x = 0.5,
             y = 0.95,
             font = dict(
                 size = 20
             )
         ),
         xaxis_title = '<b>Age</b>',
         yaxis_title = '<b>Sum of Duration</b>'
)

fig.show()

In [28]:
df = df.drop(columns = ['Cluster Segregation'], axis = 1)
X = df
print(X)

     Age       Job  Duration  Job k$  Gender
197   27  0.000002       1.2    2000       0
853   32  0.000001       1.8    1000       1
287   31  0.000003       4.8    3000       1
874   29  0.000001       1.2    1000       1
284   37  0.000002       2.4    2000       1
..   ...       ...       ...     ...     ...
53    31  0.000002       1.8    2000       1
1     22  0.000002       4.8    2000       0
311   25  0.000002       2.4    2000       1
315   31  0.000002       3.6    2000       1
0     67  0.000002       0.6    2000       1

[1000 rows x 5 columns]


In [29]:
eps_values = np.arange(11, 16, 2)
min_samples_values = np.arange(7, 13, 2)
param_list = list(product(eps_values, min_samples_values))

clusters = []
sil_score = []

for param in param_list:
    dbscan = DBSCAN(eps = param[0], min_samples = param[1]).fit(X)
    clusters.append(len(np.unique(dbscan.labels_)))
    sil_score.append(silhouette_score(X, dbscan.labels_))

dbscan_df = pd.DataFrame.from_records(param_list, columns = ['Eps', 'Min Samples'])
dbscan_df['Number of Clusters'] = clusters
dbscan_df['Silhouette Score'] = sil_score
dbscan_df

,Eps,Min Samples,Number of Clusters,Silhouette Score
0,11,7,5,0.982660
1,11,9,5,0.982660
2,11,11,5,0.982660
3,13,7,5,0.982660
4,13,9,5,0.982660
5,13,11,5,0.982660
6,15,7,4,0.987499
7,15,9,5,0.981328
8,15,11,5,0.981328


In [30]:
from matplotlib import Parameter
eps_values = np.arange(11, 16, 2)
min_samples_values = np.arange(7, 13, 2)
param_list = list(product(eps_values, min_samples_values))

clusters = []
sil_score = []

for param in param_list:
    dbscan = DBSCAN(eps = param[0], min_samples = param[1]).fit(X)
    clusters.append(len(np.unique(dbscan.labels_)))
   
    sil_score.append(silhouette_score(X, dbscan.labels_))
print(len(clusters))
dbscan_df = pd.DataFrame.from_records(param_list, columns = ['Eps', 'Min Samples'])
dbscan_df['Number of Clusters'] = clusters
dbscan_df['Silhouette Score'] = sil_score
dbscan_df


9


,Eps,Min Samples,Number of Clusters,Silhouette Score
0,11,7,5,0.982660
1,11,9,5,0.982660
2,11,11,5,0.982660
3,13,7,5,0.982660
4,13,9,5,0.982660
5,13,11,5,0.982660
6,15,7,4,0.987499
7,15,9,5,0.981328
8,15,11,5,0.981328


In [31]:
dbscan_df['Silhouette Score'] = dbscan_df['Silhouette Score'].astype(str)

fig = px.density_heatmap(dbscan_df, x = 'Eps', 
                         y = 'Min Samples', 
                         z = 'Silhouette Score', 
                         histfunc = 'max',
                         text_auto = '.4f',
                         color_continuous_scale = (['#29A0B1', '#167D7F', '#98D7C2', '#DDFFE7'][::-1]),
)

fig.update_layout(
    template = 'plotly_dark',
    width = 800,
    legend_title='Clusters',
    title = dict(
             text = '<b>DBSCAN Silhouette Scores</b>',
             x = 0.45,
             y = 0.95,
             font = dict(
                 size = 20
             )
         ),
         xaxis_title = '<b>Eps</b>',
         yaxis_title = '<b>Min Samples</b>'
)

fig.show()